# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1ogme9BJeHb2IZ6UREavUorF--nnxoWCYAAi8AZ4Q5jQ/edit?usp=sharing) and [grading rubric](https://docs.google.com/document/d/1XI9Yq_e-U-D3iH4jTPAtNteeP2Q9mtJ9NKbePWKeN_g/edit?usp=sharing)

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [ ]:
# All import statements needed for the project, for example:

import pandas as pd
import numpy as np
import time

import json
import pathlib
import urllib.parse
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely

import sqlalchemy as db
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base
import subprocess
import psycopg2
from geoalchemy2 import WKTElement, Geometry
from shapely.geometry import Point
import requests

from sqlalchemy import create_engine, text

pd.set_option('display.max_columns', None)


In [ ]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "nyc_zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "qmf4vTidE7TMehCHe78KV7SIM"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/resource/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "Group6Finalproject"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"  
DB_PORT = "5432" 

#DB_URL = f"postgresql+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [ ]:
# Make sure the QUERY_DIRECTORY & DATA_DIR exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()
if not (DATA_DIR/"data_cleaned").exists():    
    (DATA_DIR/"data_cleaned").mkdir()


## Part 1: Data Preprocessing

In [ ]:
def download_nyc_geojson_data(url, force=False):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / url_path
    
    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        
        ...
        
        with open(filename, "w") as f:
            json.dump(..., f)
        print(f"Done downloading {url}.")

    else:
        print(f"Reading from {filename}...")

    return filename

## Zillow Data

In [ ]:
def load_and_clean_zillow_data():
    df = pd.read_csv(ZILLOW_DATA_FILE)
    #Only need New York
    df = df[df['City']=='New York']
    #Remove unneeded columns
    df.drop(columns=['RegionID', 'SizeRank','RegionType','StateName','State','City','Metro','CountyName'],inplace=True)
    
    # Normalize column names used later
    column_name_mapping ={
    '2023-01-31': 'Jan_2023', '2023-02-28': 'Feb_2023', '2023-03-31': 'Mar_2023',
    '2023-04-30': 'Apr_2023', '2023-05-31': 'May_2023', '2023-06-30': 'Jun_2023',
    '2023-07-31': 'Jul_2023', '2023-08-31': 'Aug_2023', '2023-09-30': 'Sep_2023',
    '2023-10-31': 'Oct_2023', '2023-11-30': 'Nov_2023', '2023-12-31': 'Dec_2023',
    '2024-01-31': 'Jan_2024', 
    }
    
    df.rename(columns=column_name_mapping, inplace=True) 
    df.rename(columns={'RegionName': 'zip_code'}, inplace=True)
    
    for col in df.columns:
        if col != 'zip_code':
            df[col] = pd.to_numeric(df[col])
    df['zip_code'] = df['zip_code'].astype("string")
    df1 = df.copy()
    
    
    df1['mean'] = df1.iloc[:, 1:].mean(axis=1, skipna=True)
    
    clean_columns = ['zip_code', 'Mar_2023', 'Apr_2023', 'May_2023', 
                 'Jun_2023', 'Jul_2023', 'Aug_2023', 'Sep_2023', 'Oct_2023',
                 'Nov_2023', 'Dec_2023', 'Jan_2024','mean']
    df1 = df1[clean_columns]
    
    df1.iloc[:,1:] = df1.iloc[:,1:].fillna(df.iloc[:,1:].mean())
    
    return df1

In [ ]:
df_zillow_data = load_and_clean_zillow_data()

In [ ]:
df_zillow_data

## Zipcode Data

In [ ]:
def load_and_clean_zipcodes(zipcode_datafile):
    zipcode_data = gpd.read_file(zipcode_datafile)
    zipcode_data = zipcode_data[['ZIPCODE', 'geometry']]
    zipcode_data = zipcode_data.to_crs(epsg=4326)
    zipcode_data = zipcode_data.rename(columns={'ZIPCODE': 'zipcode'})
    zipcode_data = zipcode_data.dropna()
    
    return zipcode_data

In [ ]:
geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)

In [ ]:
geodf_zipcode_data

## Tree Data

In [ ]:
def download_and_clean_tree_data():
    url=f"{BASE_NYC_DATA_URL}{NYC_DATA_TREES}?$$app_token={NYC_DATA_APP_TOKEN}&$limit=10000"
    filename=download_nyc_geojson_data(url)
    df=gpd.read_file(filename)
    
    # To keep the necessary columns
    df_selected=df[['tree_id','zipcode','address','health','zip_city','spc_common','status','sidewalk','borocode','block_id','geometry']]
    
    
    # Drop rows with missing values
    df_selected=df_selected.dropna()
    
    # Convert column names to lowercase
    df_selected.columns = [col.lower() for col in df_selected.columns]
    
    # Change the SRID to a specific value (EPSG 4326 - WGS 84)
    df_selected = df_selected.to_crs(epsg=4326)
    
    
    
    return df_selected

In [ ]:
geodf_tree_data = download_and_clean_tree_data()

In [ ]:
geodf_tree_data

## NYC Open Data

In [ ]:
def download_and_clean_311_data():
    
    BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/resource/"
    NYC_DATA_311 = "erm2-nwe9.json"
    
    
    data_url = BASE_NYC_DATA_URL + NYC_DATA_311
    columns = "unique_key,created_date,closed_date,complaint_type,incident_zip,latitude,longitude"
    start_date = "2023-03-01"
    end_date = "2024-02-29"
    limit = 50000
    offset = 0
    total_rows = 0
    headers = {"X-App-Token": NYC_DATA_APP_TOKEN}
    all_data = []
    
    while True:
        query = f"$select={columns}&$where=created_date >= '{start_date}T00:00:00.000' AND created_date <= '{end_date}T23:59:59.999' AND latitude IS NOT NULL&$limit={limit}&$offset={offset}"
        paginated_url = f"{data_url}?{query}"
        response = requests.get(paginated_url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data:
                all_data.extend(data)
                total_rows += len(data)
                offset += limit
            else:
                break
        else:
            break
    # Convert to DataFrame
    df = pd.DataFrame(all_data)
    df = df.rename(columns={'incident_zip':'zip_code'})
    # Convert to GeoDataFrame
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326"
    )
    return gdf

In [ ]:
geodf_311_data = download_and_clean_311_data()

In [ ]:
geodf_311_data

In [ ]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

In [ ]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

In [ ]:
geodf_311_data.info()

In [ ]:
geodf_311_data.head()

In [ ]:
geodf_tree_data.info()

In [ ]:
geodf_tree_data.head()

In [ ]:
df_zillow_data.info()

In [ ]:
df_zillow_data.head()

## Part 2: Storing Data

In [ ]:
def setup_new_postgis_database(username, db_name, password):
    conn = psycopg2.connect(
        dbname=db_name, 
        user=username,
        password='postgres',  
        host='localhost',   
        port='5432'         
    )
    cursor = conn.cursor()
    cursor.close()
    conn.commit()
    conn.close()

In [ ]:
setup_new_postgis_database(DB_USER, DB_NAME)

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [ ]:
# If using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables.
# You may be creating more tables depending on how you're setting up your constraints/relationships
# or if you're completing the extra credit.

ZIPCODE_SCHEMA = """
    CREATE TABLE IF NOT EXISTS zip_codes (
    id INTEGER PRIMARY KEY,
    zipcode CHAR(5),
    geometry GEOMETRY (Polygon,4326)
);

"""

NYC_311_SCHEMA = """
CREATE TABLE IF NOT EXISTS NYC_311 (
    unique_key VARCHAR(50) PRIMARY KEY,
    created_date VARCHAR(50),
    closed_date VARCHAR(50),
    complaint_type VARCHAR(100),
    zip_code VARCHAR(5),
    latitude VARCHAR(50),
    longitude VARCHAR(50),
    geometry GEOMETRY(POINT, 4326)
);

"""

NYC_TREE_SCHEMA = """
DROP TABLE IF EXISTS trees;
CREATE TABLE IF NOT EXISTS trees (
    tree_id VARCHAR(50) PRIMARY KEY,
    zipcode VARCHAR(10),
    address VARCHAR(255),
    health VARCHAR(255), 
    zip_city VARCHAR(255), 
    spc_common VARCHAR(255),
    status VARCHAR(255),
    sidewalk VARCHAR(255),
    borocode VARCHAR(10),
    block_id VARCHAR(255),
    geometry GEOMETRY (Point,4326)
);
"""

ZILLOW_SCHEMA = """
CREATE TABLE IF NOT EXISTS rents
(
    id INTEGER PRIMARY KEY,
    zip_code CHAR(5),
    Jan_2023 FLOAT,
    Feb_2023 FLOAT,
    Mar_2023 FLOAT,
    Apr_2023 FLOAT,
    May_2023 FLOAT,
    Jun_2023 FLOAT,
    Jul_2023 FLOAT,
    Aug_2023 FLOAT,
    Sep_2023 FLOAT,
    Oct_2023 FLOAT,
    Nov_2023 FLOAT,
    Dec_2023 FLOAT,
    Jan_2024 FLOAT,
    mean FLOAT
);
"""

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [ ]:
conn = psycopg2.connect(f"dbname={DB_NAME} user={DB_USER} password={DB_PASSWORD}")
cur = conn.cursor()
with cur:
    cur.execute(ZIPCODE_SCHEMA)
    cur.execute(NYC_311_SCHEMA)
    cur.execute(NYC_TREE_SCHEMA)
    cur.execute(ZILLOW_SCHEMA)

conn.commit()

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
def write_dataframes_to_table(tablename_to_dataframe):
    # write INSERT statements or use pandas/geopandas to write SQL
    engine = create_engine(DB_URL)
    
    for tablename, dataframe in tablename_to_dataframe.items():
        if 'geometry' in dataframe.columns:
            temp_df = dataframe.copy()
            temp_df['the_geom'] = temp_df['geometry'].apply(lambda geom: WKTElement(geom.wkt, srid=4326) if geom is not None else None)
            temp_df.drop('geometry',axis=1,inplace=True)
            temp_df.to_sql(tablename, engine, if_exists='replace', index=False, 
                           dtype={'the_geom': Geometry('GEOMETRY', srid=4326)}) 
            
        else:
            dataframe.to_sql(tablename, engine, if_exists='replace', index=False)

In [ ]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [ ]:
write_dataframes_to_table(tablename_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
def write_query_to_file(query, outfile):
    with open(outfile, "w") as file:
        file.write(query)

### Query 1: Incidents per Zip Code 

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "complains_per_Zip_Code.sql"

QUERY_1 = """
SELECT zip_code, COUNT(*) AS complaint_count
FROM complaints
WHERE created_date >= '2023-03-01' AND created_date <= '2024-02-29'
GROUP BY zip_code
ORDER BY complaint_count DESC;
"""


In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Query 2: Green Areas by Zip Code

In [ ]:
QUERY_2_FILENAME = QUERY_DIR / "Top10_Zip_Codes_by_trees.sql"

QUERY_2 = """
SELECT zipcode, COUNT(*) AS tree_count
FROM trees
GROUP BY zipcode
ORDER BY tree_count DESC
LIMIT 10;
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_2))
    for row in result:
        print(row)

('10306', 2355)
('11230', 1545)
('10466', 1400)
('11375', 1215)
('11215', 1172)
('10312', 1115)
('11426', 1107)
('11105', 1056)
('11218', 1011)
('11372', 970)


In [ ]:
write_query_to_file(QUERY_2, QUERY_2_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)